In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Essential_Thrombocythemia/GSE103237'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene and miRNA expression profiles in Polycythemia Vera and Essential Thrombocythemia according to CALR and JAK2 mutations [GEP]"
!Series_summary	"Polycythemia vera (PV) and essential thrombocythemia (ET) are Philadelphia-negative myeloproliferative neoplasms (MPNs) characterized by erythrocytosis and thrombocytosis, respectively. Approximately 95% of PV and 50–70% of ET patients harbour the V617F mutation in the exon 14 of JAK2 gene, while about 20-30% of ET patients carry CALRins5 or CALRdel52 mutations. These ET CARL-mutated subjects show higher platelet count and lower thrombotic risk compared to JAK2-mutated patients. Here we showed that CALR-mutated and JAK2V617F-positive CD34+ cells have different gene and miRNA expression profiles. Indeed, we highlighted several pathways differentially activated between JAK2V617F- and CALR-mutated progenitors, i.e. mTOR, MAPK/PI3K and MYC pathways. Furthermore, we unveiled that the expression of several ge

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

is_gene_available = True
trait_row = 3
age_row = None
gender_row = 1

# Define the functions to convert values for each variable
def convert_trait(value):
    if isinstance(value, str):
        val = value.split(':')[-1].strip().lower()
        if val == 'et':
            return 1
        elif val in ['pv', 'healthy control']:
            return 0
    return None

def convert_age(value):
    if value is None or value == '':
        return None
    try:
        return float(value.split(':')[-1].strip())
    except ValueError:
        return None

def convert_gender(value):
    if isinstance(value, str):
        val = value.split(':')[-1].strip().lower()
        if val == 'm':
            return 1
        elif val == 'f':
            return 0
    return None

# Save cohort information
save_cohort_info('GSE103237', './preprocessed/Essential_Thrombocythemia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Essential_Thrombocythemia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Essential_Thrombocythemia/trait_data/GSE103237.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2758679': [1, 1], 'GSM2758680': [1, 0], 'GSM2758681': [1, 1], 'GSM2758682': [1, 1], 'GSM2758683': [1, 1], 'GSM2758684': [1, 1], 'GSM2758685': [1, 1], 'GSM2758686': [1, 0], 'GSM2758687': [1, 0], 'GSM2758688': [1, 0], 'GSM2758689': [1, 0], 'GSM2758690': [1, 1], 'GSM2758691': [1, 0], 'GSM2758692': [1, 1], 'GSM2758693': [1, 0], 'GSM2758694': [1, 1], 'GSM2758695': [1, 1], 'GSM2758696': [1, 0], 'GSM2758697': [1, 0], 'GSM2758698': [1, 0], 'GSM2758699': [1, 0], 'GSM2758700': [1, 0], 'GSM2758701': [1, 0], 'GSM2758702': [1, 1], 'GSM2758703': [0, 0], 'GSM2758704': [0, 0], 'GSM2758705': [0, 1], 'GSM2758706': [0, 1], 'GSM2758707': [0, 1], 'GSM2758708': [0, 1], 'GSM2758709': [0, 0], 'GSM2758710': [0, 1], 'GSM2758711': [0, 1], 'GSM2758712': [0, 1], 'GSM2758713': [0, 0], 'GSM2758714': [0, 1], 'GSM2758715': [0, 1], 'GSM2758716': [0, 1], 'GSM2758717': [0, 0], 'GSM2758718': [0, 1], 'GSM2758719': [0, 0], 'GSM2758720': [0, 0], 'GSM2758721': [0, 0], 'GSM2758722': [0, 0], 'GSM2758723': [0, 1], 'GSM27587

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /DEF=g21264

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate keys from the annotation preview
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Essential_Thrombocythemia/gene_data/GSE103237.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Essential_Thrombocythemia')

# 4. Save the cohort information.
# Use the original merged_data for the metadata, not the unbiased_merged_data variable
save_cohort_info('GSE103237', './preprocessed/Essential_Thrombocythemia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Essential_Thrombocythemia/GSE103237.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Essential_Thrombocythemia', the least common label is '1.0' with 24 occurrences. This represents 48.00% of the dataset.
The distribution of the feature 'Essential_Thrombocythemia' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 23 occurrences. This represents 46.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

